# Proceso de Limpieza de Datos de Arriendos en Puerto Montt

Este notebook toma los datos brutos extraídos por el script de web scraping (`arriendos_puerto_montt.xlsx`) y realiza un proceso completo de limpieza y transformación. El objetivo es preparar un dataset limpio y estructurado (`arriendos_puerto_montt_limpio.xlsx`) listo para ser analizado y visualizado en Power BI.

In [1]:
%load_ext kedro.ipython
%reload_kedro

import pandas as pd  # noqa
import numpy as np  # noqa
import re  # noqa
import datetime  # noqa
from rentaptomonttkedro.pipelines.data_processing.nodes import get_current_uf_value  # noqa
from unidecode import unidecode  # noqa

print("Librerías importadas y contexto de Kedro cargado correctamente.")

[09/27/25 23:33:20] INFO     Using                                                                  ]8;id=315888;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=897618;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\framework\project\__init__.py#270\270]8;;\
                             'c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-pack                
                             ages\kedro\framework\project\rich_logging.yml' as logging                             
                             configuration.                                                                        

[09/27/25 23:33:21] INFO     Registered line magic '%reload_kedro'                                   ]8;id=830451;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=229661;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=666481;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=719957;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=248102;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=416170;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py#180\180]8;;\
                             c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/27/25 23:33:21] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=869918;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=343068;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

...

                    INFO     Resolved project path as:                                              ]8;id=83965;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=667408;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py#180\180]8;;\
                             C:\Users\javie\Documents\GitHub\RentaPtoMonttKedro.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/27/25 23:33:23] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=119129;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=120164;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/27/25 23:33:24] INFO     Kedro project RentaPtoMonttKedro                                       ]8;id=516222;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=394413;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=421778;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=744200;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\ipython\__init__.py#147\147]8;;\
                             'pipelines'                                                                           

Librerías importadas y contexto de Kedro cargado correctamente.


## 1. Carga de Datos

Cargamos el archivo Excel con los datos en crudo usando el catálogo de Kedro.

In [2]:
# --- Cargar los Datos usando el Catálogo de Kedro ---
# El 'catalog' fue definido globalmente por %reload_kedro
# y usa la configuración de 'conf/base/catalog.yml'.

# El nombre "raw_arriendos" debe coincidir con el definido en catalog.yml
df = catalog.load("raw_arriendos")  # type: ignore
print("Dataset 'raw_arriendos' cargado exitosamente desde el catálogo.")

                    INFO     Loading data from raw_arriendos (ExcelDataset)...                 ]8;id=19161;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=848452;file://c:\Users\javie\Documents\GitHub\RentaPtoMonttKedro\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

Dataset 'raw_arriendos' cargado exitosamente desde el catálogo.


## 2. Exploración Inicial

Visualizamos las primeras filas del DataFrame para entender la estructura y el formato de los datos antes de la limpieza.

In [3]:
# Mostramos las primeras 5 filas para entender la estructura inicial
df.head(5)

,Tipo_de_hogar,Precio,Atributos,Ubicacion
0,Local en arriendo,UF\n25,3 banos\n625 m2 utiles,"Antonio Varas 986, Puerto Montt, Centro De Pue..."
1,Departamento en arriendo,$\n450.000,3 dormitorios\n1 bano\n42 m2 utiles,"Volcan Corcovado, Puerto Montt, La Paloma, Pue..."
2,Casa en arriendo,$\n750.000,3 dormitorios\n2 banos\n100 m2 utiles,"Laguna Del Laja 4904, Puerto Montt, Puerto Montt"
3,Departamento en arriendo,$\n500.000,2 dormitorios\n1 bano\n50 m2 utiles,"Impecable Estado / Sargento Silva, Puerto Montt"
4,Casa en arriendo,$\n850.000,3 dormitorios\n2 banos\n196 m2 utiles,Casa En Parcela Esquina A Orilla De Carretera ...


## 3. Obtención del Valor de la UF

Para estandarizar los precios, necesitamos convertir los valores que están en UF a CLP. Para ello, llamamos a nuestra función del pipeline que consulta el valor actualizado de la UF.

In [4]:
# --- Obtener el Valor de la UF ---
print("Obteniendo valor de la UF...")
uf_actual = get_current_uf_value()
fecha = datetime.datetime.now()

if uf_actual:
    uf_actual = float(uf_actual)
    print(f"Valor de la UF para el {fecha.strftime('%d-%m-%Y')}: ${uf_actual:,.2f}")
else:
    # Si la API falla, usamos un valor de respaldo razonable
    uf_actual = 40000  
    print(f"Advertencia: No se pudo obtener la UF. Se usará un valor por defecto de ${uf_actual:,.0f}")

Obteniendo valor de la UF...
Obteniendo el valor actual de la UF desde mindicador.cl...
UF obtenida al 27-09-2025 es: $39,485.65 pesos.
Valor de la UF para el 27-09-2025: $39,485.65


## 4. Limpieza y Conversión de Precios

Definimos una función para procesar la columna `Precio`. Esta función es capaz de:
- Identificar si un precio está en UF o CLP.
- Convertir los precios en UF a CLP multiplicándolos por el valor actual.
- Extraer únicamente los números de los precios en CLP, eliminando símbolos como `$` o puntos.

Luego, aplicamos esta función para crear una nueva columna `Precio_CLP` con todos los valores estandarizados.

In [5]:
def clean_price(price_str, uf_value):
    """
    Limpia una cadena de texto de precio, la convierte a CLP.
    Si el precio está en UF, lo convierte usando el valor de la UF.
    Si está en CLP, extrae solo los números.
    """
    price_str = str(price_str).strip()

    # Caso 1: Precio en UF
    if "uf" in price_str.lower():
        # Extraemos todos los números para manejar rangos (ej: "UF 20 - 22")
        numbers = re.findall(r"(\d+\.?\d*)", price_str)
        if numbers:
            try:
                numeric_values = [float(n) for n in numbers]
                uf_price_avg = sum(numeric_values) / len(numeric_values)
                return round(uf_price_avg * uf_value)
            except (ValueError, ZeroDivisionError):
                return np.nan
        return np.nan
    
    # Caso 2: Precio en CLP
    else:
        numbers_only = re.sub(r"\D", "", price_str)
        if numbers_only:
            try:
                return int(numbers_only)
            except ValueError:
                return np.nan
        return np.nan

# Aplicamos la función para crear la nueva columna 'Precio_CLP'
df['Precio_CLP'] = df['Precio'].apply(lambda x: clean_price(x, uf_actual))

# Verificamos el resultado
print("Columna 'Precio_CLP' creada.")
df[['Precio', 'Precio_CLP']].head(10)

Columna 'Precio_CLP' creada.


,Precio,Precio_CLP
0,UF\n25,987141
1,$\n450.000,450000
2,$\n750.000,750000
3,$\n500.000,500000
4,$\n850.000,850000
5,$\n550.000,550000
6,$\n470.000,470000
7,$\n450.000,450000
8,"UF\n18\n,\n52",1381998
9,$\n600.000,600000


## 5. Limpieza de Columnas de Texto

Normalizamos las columnas de texto principales (`Tipo_de_hogar`) para facilitar el análisis. Esto incluye:
- Eliminar tildes y caracteres especiales.
- Estandarizar el contenido, como quitar la frase "en arriendo" del tipo de hogar.

In [6]:
# Normalizamos texto: quitamos tildes y caracteres especiales
for col in ['Ubicacion', 'Tipo_de_hogar']:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: unidecode(str(x)))

# Estandarizamos la columna 'Tipo_de_hogar'
df['Tipo_de_hogar'] = df['Tipo_de_hogar'].str.replace(" en arriendo", "", case=False).str.strip()

print("Columnas de texto limpiadas.")

# Verificamos los cambios
df[['Tipo_de_hogar']].head()

Columnas de texto limpiadas.


,Tipo_de_hogar
0,Local
1,Departamento
2,Casa
3,Departamento
4,Casa


## 6. Extracción de Atributos

La columna `Atributos` contiene información valiosa (dormitorios, baños, superficie) en una sola cadena de texto. Usamos expresiones regulares para extraer cada uno de estos valores en columnas numéricas separadas, lo que permitirá realizar cálculos y filtros sobre ellos.

In [7]:
df['Atributos'] = df['Atributos'].astype(str)

# Usamos expresiones regulares para extraer la información de la columna 'Atributos'
df['Dormitorios'] = df['Atributos'].str.extract(r'(\d+)\s*dormitorio', flags=re.IGNORECASE).fillna(0)
df['Banos'] = df['Atributos'].str.extract(r'(\d+)\s*bano', flags=re.IGNORECASE).fillna(0)
df['Superficie_m2'] = df['Atributos'].str.extract(r'(\d+)\s*m2', flags=re.IGNORECASE).fillna(0)
df['Privados'] = df['Atributos'].str.extract(r'(\d+)\s*privado', flags=re.IGNORECASE).fillna(0)

# Convertimos las nuevas columnas a tipo numérico (entero)
df[['Dormitorios', 'Banos', 'Superficie_m2', 'Privados']] = df[['Dormitorios', 'Banos', 'Superficie_m2', 'Privados']].astype(int)

print("Atributos extraídos en nuevas columnas.")
df[['Atributos', 'Dormitorios', 'Banos', 'Superficie_m2']].head()

Atributos extraídos en nuevas columnas.


,Atributos,Dormitorios,Banos,Superficie_m2
0,3 banos\n625 m2 utiles,0,3,625
1,3 dormitorios\n1 bano\n42 m2 utiles,3,1,42
2,3 dormitorios\n2 banos\n100 m2 utiles,3,2,100
3,2 dormitorios\n1 bano\n50 m2 utiles,2,1,50
4,3 dormitorios\n2 banos\n196 m2 utiles,3,2,196


## 7. Selección Final y Limpieza de Nulos

Creamos un nuevo DataFrame `df_limpio` que contiene únicamente las columnas que nos interesan para el análisis final. Además, eliminamos cualquier fila donde no se haya podido calcular un precio válido (`Precio_CLP`).

In [8]:
# Definimos las columnas que queremos en nuestro dataset final
columnas_finales = [
    'Tipo_de_hogar',
    'Precio',
    'Atributos',
    'Ubicacion',
    'Precio_CLP',
    'Dormitorios',
    'Banos',
    'Privados',
    'Superficie_m2'
]

# Creamos el DataFrame limpio con solo las columnas deseadas
df_limpio = df[columnas_finales].copy()

# Eliminamos filas donde no se pudo calcular el precio en CLP o es cero
df_limpio.dropna(subset=['Precio_CLP'], inplace=True)
df_limpio = df_limpio[df_limpio['Precio_CLP'] > 0]

print("DataFrame final creado y valores nulos eliminados.")

DataFrame final creado y valores nulos eliminados.


## 8. Verificación Final

Revisamos la información y los tipos de datos del DataFrame limpio para asegurarnos de que todo esté en el formato correcto antes de guardarlo.

In [9]:
# Usamos convert_dtypes() para que pandas asigne los tipos de datos más eficientes
df_limpio = df_limpio.convert_dtypes()

print("Tipos de datos finales:")
df_limpio.info()

Tipos de datos finales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tipo_de_hogar  597 non-null    string
 1   Precio         597 non-null    string
 2   Atributos      597 non-null    string
 3   Ubicacion      597 non-null    string
 4   Precio_CLP     597 non-null    Int64 
 5   Dormitorios    597 non-null    Int64 
 6   Banos          597 non-null    Int64 
 7   Privados       597 non-null    Int64 
 8   Superficie_m2  597 non-null    Int64 
dtypes: Int64(5), string(4)
memory usage: 45.0 KB
